In [1]:
pip install delta-spark==3.3.2 dotenv

  Using cached delta_spark-3.3.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 7.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached delta_spark-3.3.2-py3-none-any.whl (22 kB)
Using cached dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)
  Created wheel for pyspark: filename=pyspark-3.5.7-py2.py3-none-any.whl size=317907714 sha256=88f3d4812fe84ff6fb421b4a9171020024e8e2550159552ceec008c4f2da48a8
  Stored in directory: /home/jovyan/.cache/pip/wheels/58/93/ef/df0fa5d76db2fedf77481d96ab4ae9b9a77eb52fbf2a65b727
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: 

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
from datetime import datetime

In [3]:
from dotenv import load_dotenv
import os
load_dotenv('/opt/workspace/.env')
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT_DOCKER")
MINIO_ACCESS = os.getenv("MINIO_ROOT_USER")
MINIO_SECRET = os.getenv("MINIO_ROOT_PASSWORD")

In [4]:
builder = (
    SparkSession.builder.appName("SilverToGold_Delta")
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS)
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # Delta Lake
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [5]:
#=============================================
#=============================================
# DIM_LINHA             ROUTES + LINHAS
#=============================================
#=============================================

In [6]:
today = "data_ref=" + datetime.today().strftime('%Y-%m-%d')
print(today)
SILVER_LINHAS_PATH = f"s3a://silver/linhas/{today}"
SILVER_ROUTES_PATH = "s3a://silver/gtfs/routes/"
GOLD_DIM_LINHA_PATH = "s3a://gold/dim_linha/"

print(f"📂 Lendo Silver Linhas: {SILVER_LINHAS_PATH}")
print(f"📂 Lendo Silver GTFS Routes: {SILVER_ROUTES_PATH}")

data_ref=2025-11-11
📂 Lendo Silver Linhas: s3a://silver/linhas/data_ref=2025-11-11
📂 Lendo Silver GTFS Routes: s3a://silver/gtfs/routes/


In [7]:
# Leitura
df_api = spark.read.format("delta").load(SILVER_LINHAS_PATH)
df_gtfs = spark.read.format("delta").load(SILVER_ROUTES_PATH)
df_api = df_api.alias("api")
df_gtfs = df_gtfs.alias("gtfs")

df_api.show(5)
df_gtfs.show(5)

+------------+-------------+--------+-------------+------------------+-------------------+----------+--------------------+
|codigo_linha|modo_circular|letreiro|sentido_linha|terminal_principal|terminal_secundario|  data_ref|    ingest_timestamp|
+------------+-------------+--------+-------------+------------------+-------------------+----------+--------------------+
|       35254|         true| 1012-10|            2|    jd. monte belo| term. jd. britania|2025-11-11|2025-11-11 02:46:...|
|        2486|         true| 1012-10|            1|    jd. monte belo| term. jd. britania|2025-11-11|2025-11-11 02:46:...|
|        2570|         true| 1012-21|            1|       jd. rosinha| term. jd. britania|2025-11-11|2025-11-11 02:46:...|
|       35338|         true| 1012-21|            2|       jd. rosinha| term. jd. britania|2025-11-11|2025-11-11 02:46:...|
|        2609|         true| 1014-10|            1|        morro doce|         cptm perus|2025-11-11|2025-11-11 02:46:...|
+------------+--

In [8]:
dim_linha = (
    df_api
    .filter(F.col("sentido_linha") == 1)
    .join(
        df_gtfs,
        F.col("api.letreiro") == F.col("gtfs.letreiro"),
        "left"
    )
    .select(
        F.col("api.codigo_linha"),
        F.col("api.letreiro"),
        F.col("api.modo_circular"),
        F.col("api.terminal_principal"),
        F.col("api.terminal_secundario"),
        F.col("gtfs.route_color"),
        F.current_timestamp().alias("transform_gold_timestamp")
    )
)    

dim_linha.show(10)

dim_linha.select("route_color").distinct().show()

+------------+--------+-------------+--------------------+-------------------+-----------+------------------------+
|codigo_linha|letreiro|modo_circular|  terminal_principal|terminal_secundario|route_color|transform_gold_timestamp|
+------------+--------+-------------+--------------------+-------------------+-----------+------------------------+
|        2486| 1012-10|         true|      jd. monte belo| term. jd. britania|     509E2F|    2025-11-11 18:22:...|
|        2570| 1012-21|         true|         jd. rosinha| term. jd. britania|     509E2F|    2025-11-11 18:22:...|
|        2609| 1014-10|         true|          morro doce|         cptm perus|     509E2F|    2025-11-11 18:22:...|
|        2230| 1015-10|         true|chac. maria trindade| term. jd. britania|     509E2F|    2025-11-11 18:22:...|
|         831| 1016-10|        false|  shop. center norte|      cem. do horto|     002F6C|    2025-11-11 18:22:...|
|        2141| 1017-10|        false|   conexao vl. iorio|              

In [9]:
dim_linha = (
    dim_linha
    .withColumn(
        "regiao",
        F.when(F.col("route_color") == "509E2F", "Área 1 - Noroeste")
         .when(F.col("route_color") == "002F6C", "Área 2 - Norte")
         .when(F.col("route_color") == "FFD100", "Área 3 - Nordeste")
         .when(F.col("route_color") == "DA291C", "Área 4 - Leste")
         .when(F.col("route_color") == "006341", "Área 5 - Sudeste")
         .when(F.col("route_color") == "0082BA", "Área 6 - Sul")
         .when(F.col("route_color") == "782F40", "Área 7 - Sudoeste")
         .when(F.col("route_color") == "FF671F", "Área 8 - Oeste")
         .otherwise("Outros")
    )
    .drop("route_color")
)
dim_linha.show(20)

+------------+--------+-------------+--------------------+-------------------+------------------------+-----------------+
|codigo_linha|letreiro|modo_circular|  terminal_principal|terminal_secundario|transform_gold_timestamp|           regiao|
+------------+--------+-------------+--------------------+-------------------+------------------------+-----------------+
|        2486| 1012-10|         true|      jd. monte belo| term. jd. britania|    2025-11-11 18:22:...|Área 1 - Noroeste|
|        2570| 1012-21|         true|         jd. rosinha| term. jd. britania|    2025-11-11 18:22:...|Área 1 - Noroeste|
|        2609| 1014-10|         true|          morro doce|         cptm perus|    2025-11-11 18:22:...|Área 1 - Noroeste|
|        2230| 1015-10|         true|chac. maria trindade| term. jd. britania|    2025-11-11 18:22:...|Área 1 - Noroeste|
|         831| 1016-10|        false|  shop. center norte|      cem. do horto|    2025-11-11 18:22:...|   Área 2 - Norte|
|        2141| 1017-10| 

In [10]:
# Verificar se as regiões e o letreiro/terminal correspondem com dados reais
from pyspark.sql import functions as F, Window

# Janela particionada por região, numerando cada linha dentro de cada grupo
window = Window.partitionBy("regiao").orderBy(F.rand())

# Adiciona número da linha por grupo
df_sample = (
    dim_linha
    .withColumn("row_number", F.row_number().over(window))
    .filter(F.col("row_number") <= 3)   # pega 3 de cada região
    .drop("row_number")
)

df_sample.show(50, truncate=False)

+------------+--------+-------------+---------------------+-------------------+--------------------------+-----------------+
|codigo_linha|letreiro|modo_circular|terminal_principal   |terminal_secundario|transform_gold_timestamp  |regiao           |
+------------+--------+-------------+---------------------+-------------------+--------------------------+-----------------+
|2450        |189L-10 |false        |lapa                 |conexao vl. iorio  |2025-11-11 18:22:09.405326|Área 1 - Noroeste|
|432         |8594-10 |false        |pca. ramos de azevedo|cid. d'abril       |2025-11-11 18:22:09.405326|Área 1 - Noroeste|
|412         |8050-10 |false        |lapa                 |pq. morro doce     |2025-11-11 18:22:09.405326|Área 1 - Noroeste|
|658         |971A-10 |false        |shop. d              |jd. primavera      |2025-11-11 18:22:09.405326|Área 2 - Norte   |
|2314        |N204-11 |false        |term. pq. d. pedro ii|metro tucuruvi     |2025-11-11 18:22:09.405326|Área 2 - Norte   |


In [37]:
# (
#     dim_linha
#         .write.format("delta")
#         .mode("overwrite")
#         .save(GOLD_DIM_LINHA_PATH)
# )
# print(f"dim_linha salvo em {GOLD_DIM_LINHA_PATH}")

1331

In [12]:
#=============================================
#=============================================
#=============================================
# DIM_PARADAS
#=============================================
#=============================================
#=============================================

In [13]:
SILVER_PARADAS_PATH = f"s3a://silver/paradas/{today}"
SILVER_STOPS_PATH = "s3a://silver/gtfs/stops/"
GOLD_DIM_LINHA_PATH = "s3a://gold/dim_linha/"
GOLD_DIM_PARADA_PATH = "s3a://gold/dim_parada/"

print(f"📂 Lendo Silver Linhas: {SILVER_LINHAS_PATH}")
print(f"📂 Lendo Silver GTFS Routes: {SILVER_ROUTES_PATH}")
print(f"📂 Lendo Gold dim_linhas: {GOLD_DIM_LINHA_PATH}")

📂 Lendo Silver Linhas: s3a://silver/linhas/data_ref=2025-11-11
📂 Lendo Silver GTFS Routes: s3a://silver/gtfs/routes/
📂 Lendo Gold dim_linhas: s3a://gold/dim_linha/


In [14]:
df_api = spark.read.format("delta").load(SILVER_PARADAS_PATH)
df_gtfs = spark.read.format("delta").load(SILVER_STOPS_PATH)
dim_linha = spark.read.format("delta").load(GOLD_DIM_LINHA_PATH)

df_api.show(3)
df_gtfs.show(3)
dim_linha.show(3)

+------------+-------------+-----------+--------------------+----------+----------+----------+--------------------+
|codigo_linha|codigo_parada|nome_parada|            endereco|  latitude| longitude|  data_ref|    ingest_timestamp|
+------------+-------------+-----------+--------------------+----------+----------+----------+--------------------+
|        2497|    640000381|guerino c/b|r coracao de bugr...| -23.48455|-46.723093|2025-11-11|2025-11-11 02:47:...|
|        2497|    640000382|guerino b/c|av miguel de cast...|-23.484536|-46.722711|2025-11-11|2025-11-11 02:47:...|
|        2559|    640000360|  pinel b/c|r mal. mendes de ...|-23.485948|-46.724827|2025-11-11|2025-11-11 02:47:...|
+------------+-------------+-----------+--------------------+----------+----------+----------+--------------------+
only showing top 3 rows

+---------+-------------+----------+----------+
|id_parada|  nome_parada|  latitude| longitude|
+---------+-------------+----------+----------+
|    18848|     cli

In [33]:
# Distintos da API (paradas únicas)
total_api_distinct = df_api.select("codigo_parada").distinct().count()

# Total de linhas originais (todas as ocorrências, repetidas)
total_api_raw = df_api.count()

# Quantos matches exatos de parada existem (deduplicando no join)
matches = (
    df_api.select("codigo_parada").distinct()
    .join(
        df_gtfs.select("id_parada").distinct(),
        F.col("codigo_parada") == F.col("id_parada"),
        "inner"
    )
    .count()
)

print(f"API (bruto): {total_api_raw:,}")
print(f"API (distintos): {total_api_distinct:,}")
print(f"GTFS (distintos): {df_gtfs.select('id_parada').distinct().count():,}")
print(f"Matches exatos: {matches:,}")
print(f"Match rate: {matches / total_api_distinct * 100:.2f}%")

API (bruto): 4,925
API (distintos): 364
GTFS (distintos): 22,050
Matches exatos: 364
Match rate: 100.00%


In [45]:
dim_parada = (
    df_gtfs.alias("g")
    .join(
        df_api.select("codigo_parada").distinct().alias("a"),
        F.col("g.id_parada") == F.col("a.codigo_parada"),
        "left"
    )
    .withColumn("is_corredor", F.col("a.codigo_parada").isNotNull())
    .select(
        F.col("g.id_parada").alias("codigo_parada"),
        F.col("g.nome_parada"),
        F.col("g.latitude"),
        F.col("g.longitude"),
        F.col("is_corredor"),
        F.current_timestamp().alias("transform_gold_timestamp")
    )
)

total = dim_parada.count()
corredor = dim_parada.filter(F.col("is_corredor") == True).count()
print(f"🚏 Paradas em corredores: {corredor}/{total} ({corredor/total*100:.2f}%)")

🚏 Paradas em corredores: 364/22050 (1.65%)


In [87]:
# Paths
SILVER_STOPS = "s3a://silver/gtfs/stops/"
SILVER_STOP_TIMES = "s3a://silver/gtfs/stop_times/"
SILVER_TRIPS = "s3a://silver/gtfs/trips/"
SILVER_PARADAS_API = "s3a://silver/paradas/"
GOLD_DIM_PARADA = "s3a://gold/dim_parada/"

# Leitura das tabelas
stops = spark.read.format("delta").load(SILVER_STOPS)
stop_times = spark.read.format("delta").load(SILVER_STOP_TIMES)
trips = spark.read.format("delta").load(SILVER_TRIPS)
paradas_api = spark.read.format("delta").load(SILVER_PARADAS_API)

# 1. join stop_times -> trips para saber qual linha atende cada parada
linha_por_parada = (
    stop_times
    .join(trips, "trip_id", "left")
    .select(
        F.col("stop_id").alias("codigo_parada"),
        F.col("letreiro")
    )
    # Filtrar por IDs numéricos (paradas de onibus)
    .filter(F.col("codigo_parada").rlike("^[0-9]+$"))
    # Remover linhas de metrô e CPTM
    .filter(~F.lower(F.col("letreiro")).rlike("metr|cptm"))
    .distinct()
)

# 2. agrega lista e contagem de linhas por parada
agg_parada = (
    linha_por_parada
    .groupBy("codigo_parada")
    .agg(
        F.collect_set("letreiro").alias("linhas_atendidas"),
        F.size(F.collect_set("letreiro")).alias("num_linhas_atendidas")
    )
)

# 3. base GTFS (stops)
dim_base = (
    stops
    .select(
        F.col("id_parada").alias("codigo_parada"),
        F.col("nome_parada"),
        F.col("latitude"),
        F.col("longitude")
    )
)

# 4️. join com paradas da API (para marcar se é corredor)
dim_final = (
    dim_base.alias("gtfs")
    .join(
        paradas_api.select("codigo_parada").distinct().alias("api"),
        F.col("gtfs.codigo_parada") == F.col("api.codigo_parada"),
        "left"
    )
    .join(agg_parada.alias("agg"), "codigo_parada", "left")
    .withColumn("is_corredor", F.col("api.codigo_parada").isNotNull())
    .withColumn(
        "tipo_parada",
        F.when(F.col("num_linhas_atendidas").isNull() | (F.col("num_linhas_atendidas") == 0),
               F.lit("metro/cptm"))
         .otherwise(F.lit("onibus"))
    )
    .filter(F.col("tipo_parada") == "onibus")
    # normaliza os nomes
    .withColumn("nome_parada", F.initcap(F.col("nome_parada")))
    # colunas finais
    .select(
        F.col("gtfs.codigo_parada").cast("int"),
        "nome_parada",
        "latitude",
        "longitude",
        "num_linhas_atendidas",
        "is_corredor",
        "linhas_atendidas",
        F.current_timestamp().alias("transform_gold_timestamp")
    )
)
# 5. Escrever na gold
(
    dim_final.write
    .format("delta")
    .mode("overwrite")
    .save(GOLD_DIM_PARADA)
)

# Logs básicos
total = dim_final.count()
corredor = dim_final.filter(F.col("is_corredor") == True).count()
print(f"✅ Gold: {total} paradas totais | 🚌 {corredor} em corredores ({corredor/total*100:.2f}%)")
print("💾 Gravado em:", GOLD_DIM_PARADA)

dim_final.show(50)

✅ Gold: 21853 paradas totais | 🚌 364 em corredores (1.67%)
💾 Gravado em: s3a://gold/dim_parada/
+-------------+--------------------+----------+----------+--------------------+-----------+--------------------+------------------------+
|codigo_parada|         nome_parada|  latitude| longitude|num_linhas_atendidas|is_corredor|    linhas_atendidas|transform_gold_timestamp|
+-------------+--------------------+----------+----------+--------------------+-----------+--------------------+------------------------+
|        19070| Via Transversal Sul|-23.580717|-46.794016|                   1|      false|           [809H-10]|    2025-11-12 00:18:...|
|        19071|Av. Dos Ourives, 780|-23.655442|-46.607946|                   5|      false|[519L-10, 4491-21...|    2025-11-12 00:18:...|
|        19072|Av. Dos Ourives, 780|-23.655265|-46.607882|                   5|      false|[519L-10, 4491-21...|    2025-11-12 00:18:...|
|        19073|Rod. Arao Sahm, 2...|-23.373502|-46.575335|                  

In [88]:
dim_final.printSchema()

root
 |-- codigo_parada: integer (nullable = true)
 |-- nome_parada: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- num_linhas_atendidas: integer (nullable = true)
 |-- is_corredor: boolean (nullable = false)
 |-- linhas_atendidas: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- transform_gold_timestamp: timestamp (nullable = false)

